In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt

from src.utils import print_yaml, bootstrap_sampling, interpolate_unique
from src.save_load import loadAudit, savePlot, loadFbdStudy, loadTargetSignals, loadShadowModelSignals
from LeakPro.leakpro.attacks.mia_attacks.lira import lira_vectorized

In [ ]:
# -------------------------- #
#   Load fbd study results   #
# -------------------------- #
study_name = "cifar10-resnet-fbd-815409b641"
study_path = os.path.join("study", study_name)
metadata = []
gtl_probs = []
resc_logits = []

metadata, fbd_trial_results, gtl_probs, resc_logits = loadFbdStudy(study_name, metadata=True, gtl=True, logits=True)
target_folder = metadata["study"]["target_folder"]

In [ ]:
# -------------------------------- #
#   Load baseline target signals   #
# -------------------------------- #
target_folder = metadata["study"]["target_folder"]
baseline_target_logits, baseline_targetinmask, baseline_target_metadata = loadTargetSignals(target_folder)
# Rescale and calc gtl for baseline target

In [ ]:
# ----------------------------- #
#   Load shadow model signals   #
# ----------------------------- #
shadow_logits_all, shadow_inmask_all = loadShadowModelSignals(target_folder)
# Rescale and calc gtl for shadow models

In [ ]:
# Audit the baseline target with LiRA
# Audit the baseline target with RMIA

# Audit all weighted models with lira

In [ ]:
# --------------------------------- #
#   Visualize Model Study Results   #
# --------------------------------- #
accuracy = [res.accuracy for res in fbd_trial_results]
tau_rmia = [res.tau for res in fbd_trial_results]

# Plot
fig, ax = plt.subplots()
ax.scatter(tau_rmia, accuracy)

plot_title = "cifar10_resnet18_FbD_study"
ax.set_xlabel("tau@0.1")
ax.set_ylabel("Accuracy")
ax.set_title(plot_title)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()



In [ ]:
# -------------------------- #
#   Visualize LiRA results   #
# -------------------------- #

In [ ]:
# --------------------------- #
#   Setup for bootstrapping   #
# --------------------------- #
# Shadow models used
num_sm = audit_cfg["audit"]["attack_list"][0]["num_shadow_models"]
# Samples for bootstrapping
no_samples = 100
random_target = True

In [ ]:
# ---------------------- #
#   Bootstrap sampling   #
# ---------------------- #

samples = bootstrap_sampling(num_sm, no_samples, rescaled_sm_logits, sm_in_mask, rescaled_target_logits, target_in_mask, vec_mia_fun=lira_vectorized)

unique_fpr = sorted(np.unique(np.concat([fpr for (fpr, tpr) in samples])))

ig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].plot(unique_fpr, unique_fpr, 'r--', label="random")
axs[1].loglog(unique_fpr, unique_fpr, 'r--', label="random")
for i in [0,1]:
    for (fpr, tpr) in samples:
        axs[i].plot(fpr, tpr)
    axs[i].set_xlabel("FPR")
    axs[i].set_ylabel("TPR")
    axs[i].legend()
plt.tight_layout()
plt.show()

plot_title = "cifar10_resnet18_baseline_tpr-fpr"

In [ ]:
# ---------------------- #
#   Bootstrap sampling   #
# ---------------------- #
interp_tpr = np.array([interpolate_unique(unique_fpr, fpr, tpr) for (fpr, tpr) in samples])

fig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].plot(fpr, fpr, 'r--', label="random")
axs[1].loglog(fpr, fpr, 'r--', label="random")
for i in [0,1]:
    axs[i].plot(unique_fpr,np.mean(interp_tpr,axis=0))
    axs[i].fill_between(unique_fpr,np.percentile(interp_tpr,2.5,axis=0),np.percentile(interp_tpr,97.5,axis=0), alpha=0.5)
    axs[i].set_xlabel("FPR")
    axs[i].set_ylabel("TPR")
    axs[i].legend()
plt.tight_layout()
plt.show()

plot_title = "cifar10_resnet18_baseline_tpr-fpr_confidence"
savePlot(fig, plot_title, audit_dir)